In [41]:
import gym
env = gym.make('CartPole-v0')

/home/murali/.local/lib/python3.11/site-packages/gym/envs/registration.py:555: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(


In [42]:
import gym
env = gym.make("LunarLander-v2", render_mode="human")
env.action_space.seed(42)

observation, info = env.reset(seed=42)

for _ in range(1000):
    observation, reward, terminated, truncated, info = env.step(env.action_space.sample())

    if terminated or truncated:
        observation, info = env.reset()

env.close()


In [43]:
import gym
from stable_baselines3 import DQN
import numpy as np
from stable_baselines3.common.vec_env import DummyVecEnv

# Set a random seed for reproducibility
np.random.seed(42)

# Create the LunarLander-v2 environment
env = gym.make("LunarLander-v2")

# Wrap the environment with a Monitor wrapper (optional, for recording statistics)
# monitor_env = gym.wrappers.Monitor(env, "logs/", force=True)

# Wrap the environment in a DummyVecEnv (required for stable-baselines3 DQN)
# env = gym.wrappers.DummyVecEnv([lambda: monitor_env])

# Create the DQN agent
model = DQN("MlpPolicy", env, verbose=1)

# Train the agent
model.learn(total_timesteps=100000)

# Save the trained model (optional)
model.save("dqn_lunar_lander")

# Close the environment
env.close()


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 120      |
|    ep_rew_mean      | -188     |
|    exploration_rate | 0.954    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 1795     |
|    time_elapsed     | 0        |
|    total_timesteps  | 481      |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 110      |
|    ep_rew_mean      | -206     |
|    exploration_rate | 0.917    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 2400     |
|    time_elapsed     | 0        |
|    total_timesteps  | 878      |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 105      |
|    ep_rew_mean      | -222   

In [44]:
import gym
from stable_baselines3 import DQN
from stable_baselines3.common.vec_env import DummyVecEnv

# Load the trained model
model = DQN.load("dqn_lunar_lander")

# Create the LunarLander-v2 environment
env = gym.make("LunarLander-v2")

# Wrap the environment in a DummyVecEnv (required for stable-baselines3 DQN)
env = DummyVecEnv([lambda: env])

# Reset the environment to get the initial state
obs = env.reset()

# Play the game with the trained model
done = False
while not done:
    # Use the trained model to select an action based on the current state
    action, _ = model.predict(obs, deterministic=True)

    # Step the environment with the selected action
    obs, reward, done, info = env.step(action)

    # Render the environment (optional)
    env.render()

# Close the environment
env.close()


In [50]:
import gym
import numpy as np
from stable_baselines3 import PPO

# Function to preprocess the observation
def preprocess_obs(obs):
    # Extract relevant information from the observation
    return obs[:8]  # Use the first 8 elements of the observation array

# Step 1: Set up the environment
env = gym.make("LunarLander-v2")

# Step 2: Create and train the agent using PPO
model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=100000)

# Step 3: Evaluate the agent with rendering
num_episodes = 10
total_reward = 0.0

# Store the initial observation outside the loop
obs = env.reset()

for _ in range(num_episodes):
    episode_reward = 0.0

    while True:
        # Preprocess the observation
        state = preprocess_obs(obs)

        # Predict action using the model
        action, _ = model.predict(state, deterministic=True)

        # Step the environment with the predicted action
        obs, reward, done, _ = env.step(action)
        episode_reward += reward

        # Render the environment
        env.render()

        if done:
            # Reset the environment for the next episode
            obs = env.reset()
            break

    total_reward += episode_reward

mean_reward = total_reward / num_episodes
print(f"Mean reward: {mean_reward}")

# Step 4: Close the environment
env.close()


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 88.4     |
|    ep_rew_mean     | -181     |
| time/              |          |
|    fps             | 1379     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 94.1        |
|    ep_rew_mean          | -178        |
| time/                   |             |
|    fps                  | 1102        |
|    iterations           | 2           |
|    time_elapsed         | 3           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.010450903 |
|    clip_fraction        | 0.0348      |
|    clip_range           | 0.2         |
|    entropy_loss   

ValueError: Error: Unexpected observation shape (2,) for Box environment, please use (8,) or (n_env, 8) for the observation shape.